In [29]:
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score

In [2]:
os.listdir()

['.ipynb_checkpoints',
 'covid_19_data.csv',
 'eurrub.xlsx',
 'hw_4_5_TS.ipynb',
 'Lesson_5_tasks.ipynb',
 'my_BRENT2019.csv',
 'my_PJME_MW.csv',
 'Tasks.docx',
 'usdrub.xlsx',
 '~$Tasks.docx',
 'Анализ COVID 19.ipynb',
 'Временные ряды 5.ipynb']

### Task_1:

1.	Прочитайте базу my_BRENT2019.csv и перейдите к представлению ряда первыми разностями

In [3]:
df = pd.read_csv('my_BRENT2019.csv', sep=',', parse_dates=['Дата'])
df = df.rename(columns={'Дата': 'date'})
df.index = df['date']
df.drop(columns=['date'], inplace=True)
df.head()

,Значение,year,month,dayofmonth,quarter,dayofyear,weekofyear,dayofweek,is_holiday,is_prev_holiday,is_next_holiday
date,,,,,,,,,,,
2010-09-01,76.37,2010,9,1,3,244,35,2,0,0,0
2010-09-02,76.89,2010,9,2,3,245,35,3,0,0,0
2010-09-03,76.57,2010,9,3,3,246,35,4,0,0,0
2010-09-04,76.62,2010,9,4,3,247,35,5,0,0,0
2010-09-05,76.63,2010,9,5,3,248,35,6,0,1,0


In [4]:
df['dif_1'] = df['Значение'].diff(1)
df.head()

,Значение,year,month,dayofmonth,quarter,dayofyear,weekofyear,dayofweek,is_holiday,is_prev_holiday,is_next_holiday,dif_1
date,,,,,,,,,,,,
2010-09-01,76.37,2010,9,1,3,244,35,2,0,0,0,NaN
2010-09-02,76.89,2010,9,2,3,245,35,3,0,0,0,0.52
2010-09-03,76.57,2010,9,3,3,246,35,4,0,0,0,-0.32
2010-09-04,76.62,2010,9,4,3,247,35,5,0,0,0,0.05
2010-09-05,76.63,2010,9,5,3,248,35,6,0,1,0,0.01


### Task_2:

2.	Добавьте первые разности по стоимости доллара и евро к рублю (можно добавить и сами курсы валют - прошлые значения)

In [5]:
curr = [file for file in os.listdir() if 'xl' in file]
curr

['eurrub.xlsx', 'usdrub.xlsx']

In [6]:
def process_curr(path):
    d = pd.read_excel(path,  parse_dates=['data'])
    d = d.rename(columns={'data': 'date', 'curs': 'value'})
    d.index = d['date']
    d.drop(columns=['cdx', 'nominal', 'date'], inplace=True)
    return d

In [7]:
eur = process_curr(curr[0])
usd = process_curr(curr[1])

In [8]:
usd.head(3)

,value
date,
2000-01-01,27.00
2000-01-06,26.90
2000-01-07,27.23


In [9]:
eur.head(3)

,value
date,
2000-01-01,27.20
2000-01-06,27.92
2000-01-07,28.21


In [10]:
usd.rename(columns={'value': 'usd'}, inplace=True)
eur.rename(columns={'value': 'eur'}, inplace=True)

usd['usd_shift_1'] = usd['usd'].shift(1)
usd['usd_dif_1'] = usd['usd_shift_1'].diff(1)
usd.drop(columns=['usd'], inplace=True)
usd = usd[2:]


eur['eur_shift_1'] = eur['eur'].shift(1)
eur['eur_dif_1'] = eur['eur_shift_1'].diff(1)
eur.drop(columns=['eur'], inplace=True)
eur = eur[2:]

In [11]:
usd.isna().sum()

usd_shift_1    0
usd_dif_1      0
dtype: int64

In [12]:
df = pd.merge(df, usd, how='left', on='date')
df = pd.merge(df, eur, how='left', on='date')

df.head(3)

,Значение,year,month,dayofmonth,quarter,dayofyear,weekofyear,dayofweek,is_holiday,is_prev_holiday,is_next_holiday,dif_1,usd_shift_1,usd_dif_1,eur_shift_1,eur_dif_1
date,,,,,,,,,,,,,,,,
2010-09-01,76.37,2010,9,1,3,244,35,2,0,0,0,NaN,30.6640,-0.0329,39.0291,-0.0020
2010-09-02,76.89,2010,9,2,3,245,35,3,0,0,0,0.52,30.8669,0.2029,39.0127,-0.0164
2010-09-03,76.57,2010,9,3,3,246,35,4,0,0,0,-0.32,30.8001,-0.0668,39.1654,0.1527


In [13]:
d = df.copy()

In [14]:
def fill_na(name, d):
    d.loc[d[name].isna(), name] = d[name].shift(1)
    d.loc[d[name].isna(), name] = d[name].shift(1)
    return d

In [15]:
df = fill_na('usd_shift_1', df)
df = fill_na('usd_dif_1', df)
df = fill_na('eur_shift_1', df)
df = fill_na('eur_dif_1', df)
df = df[1:]
df.head(7)

,Значение,year,month,dayofmonth,quarter,dayofyear,weekofyear,dayofweek,is_holiday,is_prev_holiday,is_next_holiday,dif_1,usd_shift_1,usd_dif_1,eur_shift_1,eur_dif_1
date,,,,,,,,,,,,,,,,
2010-09-02,76.89,2010,9,2,3,245,35,3,0,0,0,0.52,30.8669,0.2029,39.0127,-0.0164
2010-09-03,76.57,2010,9,3,3,246,35,4,0,0,0,-0.32,30.8001,-0.0668,39.1654,0.1527
2010-09-04,76.62,2010,9,4,3,247,35,5,0,0,0,0.05,30.6858,-0.1143,39.2901,0.1247
2010-09-05,76.63,2010,9,5,3,248,35,6,0,1,0,0.01,30.6858,-0.1143,39.2901,0.1247
2010-09-06,76.87,2010,9,6,3,249,36,0,1,0,0,0.24,30.6858,-0.1143,39.2901,0.1247
2010-09-07,77.26,2010,9,7,3,250,36,1,0,0,1,0.39,30.6922,0.0064,39.3505,0.0604
2010-09-08,77.93,2010,9,8,3,251,36,2,0,0,0,0.67,30.5771,-0.1151,39.4720,0.1215


### Task_3:

3.	сделайте ее перрасчет (ресемплинг) в представление по неделям.

In [16]:
df = df.resample('W').mean()
df.head(3)

,Значение,year,month,dayofmonth,quarter,dayofyear,weekofyear,dayofweek,is_holiday,is_prev_holiday,is_next_holiday,dif_1,usd_shift_1,usd_dif_1,eur_shift_1,eur_dif_1
date,,,,,,,,,,,,,,,,
2010-09-05,76.677500,2010.0,9.0,3.500000,3.0,246.500000,35.0,4.500000,0.000000,0.25,0.000000,0.065000,30.759650,-0.023125,39.189575,0.096425
2010-09-12,77.673333,2010.0,9.0,8.500000,3.0,251.500000,36.0,2.500000,0.166667,0.00,0.166667,0.288333,30.742400,0.013333,39.319283,0.003567
2010-09-19,78.693333,2010.0,9.0,15.666667,3.0,258.666667,37.0,2.666667,0.000000,0.00,0.000000,-0.021667,30.773867,-0.018467,39.508300,0.166700


### Task_4:

4.	Постройте модель предсказания 1-й точки от текущей (h=1).

In [17]:
df['target'] = df['Значение'].shift(-1)
df.head(5)

,Значение,year,month,dayofmonth,quarter,dayofyear,weekofyear,dayofweek,is_holiday,is_prev_holiday,is_next_holiday,dif_1,usd_shift_1,usd_dif_1,eur_shift_1,eur_dif_1,target
date,,,,,,,,,,,,,,,,,
2010-09-05,76.677500,2010.0,9.000000,3.500000,3.000000,246.500000,35.0,4.500000,0.000000,0.25,0.000000,0.065000,30.759650,-0.023125,39.189575,0.096425,77.673333
2010-09-12,77.673333,2010.0,9.000000,8.500000,3.000000,251.500000,36.0,2.500000,0.166667,0.00,0.166667,0.288333,30.742400,0.013333,39.319283,0.003567,78.693333
2010-09-19,78.693333,2010.0,9.000000,15.666667,3.000000,258.666667,37.0,2.666667,0.000000,0.00,0.000000,-0.021667,30.773867,-0.018467,39.508300,0.166700,78.518333
2010-09-26,78.518333,2010.0,9.000000,22.666667,3.000000,265.666667,38.0,2.666667,0.000000,0.00,0.000000,0.118333,30.975133,-0.017117,40.727150,0.311700,81.211667
2010-10-03,81.211667,2010.0,9.333333,19.666667,3.333333,272.666667,39.0,2.666667,0.000000,0.00,0.000000,0.791667,30.658300,-0.149533,41.248650,0.107817,84.146667


In [18]:
df = df[:-1]

In [19]:
df.isna().sum()

Значение           1
year               1
month              1
dayofmonth         1
quarter            1
dayofyear          1
weekofyear         1
dayofweek          1
is_holiday         1
is_prev_holiday    1
is_next_holiday    1
dif_1              1
usd_shift_1        2
usd_dif_1          2
eur_shift_1        2
eur_dif_1          2
target             1
dtype: int64

In [20]:
df = df.loc[df['usd_shift_1'].notna(), :]
df = df.loc[df['target'].notna(), :]

In [21]:
df.isna().sum()

Значение           0
year               0
month              0
dayofmonth         0
quarter            0
dayofyear          0
weekofyear         0
dayofweek          0
is_holiday         0
is_prev_holiday    0
is_next_holiday    0
dif_1              0
usd_shift_1        0
usd_dif_1          0
eur_shift_1        0
eur_dif_1          0
target             0
dtype: int64

In [22]:
df.shape

(466, 17)

In [23]:
test_count = 100

y_train = df[:-100]['target']
y_test = df[-100:]['target']

X_train = df[:-100].copy()
X_test = df[-100:].copy()

X_train.drop(columns=['target'], inplace=True)
X_test.drop(columns=['target'], inplace=True)

In [41]:
model = GradientBoostingRegressor(max_depth=5, random_state=117, n_estimators=1000)
model_fitted = model.fit(X=X_train, y=y_train)

predict = model.predict(X_test)

In [43]:
r2 = r2_score(y_test, predict)
r2

0.6560498661680108

### Task_5:

5.	Сделайте вывод о поведении ошибки предсказания ряда в сравнении с прямым прогнозированием.

##### Выводы:

Не совсем понял, что с чем сравнивать, вернее что понятно, с чем - не совсем. Могу сказать, что хоть вроде 0.65 и заметное значение R2, но при других random_state этот показатель очень сильно "гуляет". Плюс оценивать качество предикта на таких горизонтах не очень информативно - значение + приращение на 1 шаг всегда будет очень близко к прошлому значению. По идее надо оценивать предикт на приращениях. Вернее смотря какие цели. Если цель получить значение - это одно, если приращение и/или направление приращения - другое. Например, если торговать нефть на бирже, то важно именно приращение, вероятно, на приращениях мерика R2 будет давать заметно хуже значения.